In [1]:
jiant_url = "https://drive.google.com/u/0/uc?id=1lFbvdomHhLNSxm37OBxGYsinWy3vYC9U&export=download"
!wget "$jiant_url" -O jiant.zip
!unzip jiant.zip -d ./
!rm jiant.zip

%cd jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./

--2021-06-22 07:43:00--  https://drive.google.com/u/0/uc?id=1lFbvdomHhLNSxm37OBxGYsinWy3vYC9U&export=download
Resolving drive.google.com (drive.google.com)... 74.125.195.101, 74.125.195.113, 74.125.195.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8fugu9pv8hloc6ot53g965p5o58fifvr/1624347750000/13675844253040106205/*/1lFbvdomHhLNSxm37OBxGYsinWy3vYC9U?e=download [following]
--2021-06-22 07:43:02--  https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8fugu9pv8hloc6ot53g965p5o58fifvr/1624347750000/13675844253040106205/*/1lFbvdomHhLNSxm37OBxGYsinWy3vYC9U?e=download
Resolving doc-04-18-docs.googleusercontent.com (doc-04-18-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-04-18-docs.googleusercontent.com (do

Obtaining file:///content/jiant
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Running setup.py develop for jiant


Restart runtime after installing libs

In [2]:
%cd /content/jiant

import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

def init_task_config(task_names):
  for task_name in task_names:
    os.makedirs("/content/tasks/configs/", exist_ok=True)
    os.makedirs(f"/content/tasks/data/{task_name}", exist_ok=True)

    py_io.write_json({
      "task": task_name,
      "paths": {
        "train": f"/content/tasks/data/{task_name}/train.jsonl",
        "val":   f"/content/tasks/data/{task_name}/val.jsonl",
      },
      "name": task_name
    }, f"/content/tasks/configs/{task_name}_config.json")

/content/jiant


In [3]:
task_names = ["ner", "semeval", "coref",  # supported by jiant 
...           "srl"]                      # additional task for other badge     
#...           "sup-squad", "ques-squad"]  # todo: define tasks in jiant 

init_task_config(task_names)

In [4]:
# contains data of all probing tasks above
task_data_url = "https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download"
!wget "$task_data_url" -O /content/task_data.zip
!unzip /content/task_data.zip -d /content/tasks/
!rm /content/task_data.zip

--2021-06-22 07:44:14--  https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download
Resolving drive.google.com (drive.google.com)... 74.125.195.139, 74.125.195.102, 74.125.195.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.195.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ct4fk5dv6m90bmqdihno9hu39n00nbee/1624347825000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download [following]
--2021-06-22 07:44:15--  https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ct4fk5dv6m90bmqdihno9hu39n00nbee/1624347825000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download
Resolving doc-04-18-docs.googleusercontent.com (doc-04-18-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-04-18-docs.googleusercontent.com (do

In [5]:
import jiant.proj.main.export_model as export_model

models = ["bert-base-uncased", "csarron/bert-base-uncased-squad-v1"]

for model in models:
  export_model.export_model(
      hf_pretrained_model_name_or_path=model,
      output_base_path=f"/content/models/{model}",
  )

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForPreTraining: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at csarron/bert-base-uncased-squad-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.

In [6]:
import jiant.shared.caching as caching
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

# Tokenize and cache each task
def tokenize(task_names):
  for task_name in task_names:
    tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
        task_config_path=f"/content/tasks/configs/{task_name}_config.json",
        hf_pretrained_model_name_or_path="csarron/bert-base-uncased-squad-v1",
        output_dir=f"/content/cache/{task_name}",
        phases=["train", "val"],
    ))

tokenize(task_names)

NerTask
  [train]: /content/tasks/data/ner/train.jsonl
  [val]: /content/tasks/data/ner/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT

SemevalTask
  [train]: /content/tasks/data/semeval/train.jsonl
  [val]: /content/tasks/data/semeval/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT



ModelArchitectures.BERT
ModelArchitectures.BERT

CorefTask
  [train]: /content/tasks/data/coref/train.jsonl
  [val]: /content/tasks/data/coref/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT

SrlTask
  [train]: /content/tasks/data/srl/train.jsonl
  [val]: /content/tasks/data/srl/val.jsonl


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT



In [7]:
row = caching.ChunkedFilesDataCache(f"/content/cache/{task_names[3]}/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)
print(row.spans)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
#print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

[  101  2006  2285  1023  1010  1996  1000 14004  1011 10556 11631  5332
  5575  1000  3864  2234  2000  2019  2203  1012   102     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
['[CLS]', 'on', 'december', '9', ',', 'the', '"', 'taipei', '-', 'ka', '##oh', '##si', '##ung', '"', 'elections', 'came', 'to', 'an', 'end', '.', '[SEP]']
[[15 15]
 [16 18]]
['came']


In [8]:
import jiant.proj.main.scripts.configurator as configurator

def create_jiant_task_config(task_name, epochs=5):
  jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
      task_config_base_path="/content/tasks/configs",
      task_cache_base_path="/content/cache",
      train_task_name_list=[task_name],
      val_task_name_list=[task_name],
      train_batch_size=8,
      eval_batch_size=2,
      epochs=epochs,
      num_gpus=1,
  ).create_config()
  os.makedirs("/content/run_configs/", exist_ok=True)
  py_io.write_json(jiant_run_config, f"/content/run_configs/{task_name}_run_config.json")
  #display.show_json(jiant_run_config)

In [9]:
# You may restart runtime when you modify the jiant source code

import jiant.proj.main.runscript as main_runscript

def run_probing_task(task_name, model_name="bert-base-uncased", num_layers=1, eval_every_steps=1000):
  run_args = main_runscript.RunConfiguration(
      jiant_task_container_config_path=f"/content/run_configs/{task_name}_run_config.json",
      output_dir=f"/content/runs/{task_name}",
      hf_pretrained_model_name_or_path=model_name,
      model_path=f"/content/models/{model_name}/model/model.p",
      model_config_path=f"/content/models/{model_name}/model/config.json",
      learning_rate=1e-5,
      eval_every_steps=eval_every_steps,
      do_train=True,
      do_val=True,
      do_save=True,
      force_overwrite=True,
      num_hidden_layers=num_layers,
  )
  return main_runscript.run_loop(run_args)

In [10]:
task_names = ["ner", "semeval", "coref",  # supported by jiant 
...           "srl"]                      # additional task for other badge     
#...           "sup-squad", "ques-squad"]  # todo: define new tasks in jiant 
models = ["bert-base-uncased", "csarron/bert-base-uncased-squad-v1"]
num_layers = list(range(1, 5))      # from 1 to 12 layers

# Hyperparameters
epochs = 5              # used in paper
eval_every_steps = 1000 # used in paper

result = "model\ttask\tlayer\tacc\n"

for model in models:
  for task in task_names:
    for n_layers in num_layers:
      create_jiant_task_config(task, epochs=epochs)
      probing_output = run_probing_task(task, model, n_layers, eval_every_steps=eval_every_steps)
      result += model + "\t" + task + "\t" + str(n_layers) + "\t" + str(probing_output[task]["metrics"]["minor"]["acc"]) + "\n"

  jiant_task_container_config_path: /content/run_configs/ner_run_config.json
  output_dir: /content/runs/ner
  hf_pretrained_model_name_or_path: bert-base-uncased
  model_path: /content/models/bert-base-uncased/model/model.p
  model_config_path: /content/models/bert-base-uncased/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 1
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 1e-05
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
Using seed: 2843033891
{
  "jiant_task_container_con

/content/jiant/jiant/proj/main/modeling/model_setup.py:177: UserWarning: The following weights were not loaded: dict_keys(['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias'])
  "The following weights were not loaded: {}".format(remainder_weights_dict.keys())


No optimizer decay for:
  encoder.embeddings.LayerNorm.weight
  encoder.embeddings.LayerNorm.bias
  encoder.encoder.layer.0.attention.self.query.bias
  encoder.encoder.layer.0.attention.self.key.bias
  encoder.encoder.layer.0.attention.self.value.bias
  encoder.encoder.layer.0.attention.output.dense.bias
  encoder.encoder.layer.0.attention.output.LayerNorm.weight
  encoder.encoder.layer.0.attention.output.LayerNorm.bias
  encoder.encoder.layer.0.intermediate.dense.bias
  encoder.encoder.layer.0.output.dense.bias
  encoder.encoder.layer.0.output.LayerNorm.weight
  encoder.encoder.layer.0.output.LayerNorm.bias
  encoder.pooler.dense.bias
  taskmodels_dict.ner.head.span_attention_extractor._global_attention._module.bias
  taskmodels_dict.ner.head.classifier.bias
Using AdamW



Loading Best



{
  "aggregated": 0.4722222222222222,
  "ner": {
    "loss": 0.6458781063556671,
    "metrics": {
      "major": 0.4722222222222222,
      "minor": {
        "acc": 0.9444444444444444,
        "f1_micro": 0.0,
        "acc_and_f1_micro": 0.4722222222222222
      }
    }
  }
}
  jiant_task_container_config_path: /content/run_configs/ner_run_config.json
  output_dir: /content/runs/ner
  hf_pretrained_model_name_or_path: bert-base-uncased
  model_path: /content/models/bert-base-uncased/model/model.p
  model_config_path: /content/models/bert-base-uncased/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 2
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 1e-05
  adam_epsilon:

...


{
  "aggregated": 0.4659090909090909,
  "srl": {
    "loss": 0.6732827126979828,
    "metrics": {
      "major": 0.4659090909090909,
      "minor": {
        "acc": 0.9318181818181818,
        "f1_micro": 0.0,
        "acc_and_f1_micro": 0.4659090909090909
      }
    }
  }
}
  jiant_task_container_config_path: /content/run_configs/srl_run_config.json
  output_dir: /content/runs/srl
  hf_pretrained_model_name_or_path: csarron/bert-base-uncased-squad-v1
  model_path: /content/models/csarron/bert-base-uncased-squad-v1/model/model.p
  model_config_path: /content/models/csarron/bert-base-uncased-squad-v1/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 3
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: Tru


Loading Best



{
  "aggregated": 0.4640151515151515,
  "srl": {
    "loss": 0.6775818765163422,
    "metrics": {
      "major": 0.4640151515151515,
      "minor": {
        "acc": 0.928030303030303,
        "f1_micro": 0.0,
        "acc_and_f1_micro": 0.4640151515151515
      }
    }
  }
}
  jiant_task_container_config_path: /content/run_configs/srl_run_config.json
  output_dir: /content/runs/srl
  hf_pretrained_model_name_or_path: csarron/bert-base-uncased-squad-v1
  model_path: /content/models/csarron/bert-base-uncased-squad-v1/model/model.p
  model_config_path: /content/models/csarron/bert-base-uncased-squad-v1/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 4
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True


Loading Best



{
  "aggregated": 0.576854066985646,
  "srl": {
    "loss": 0.6555856466293335,
    "metrics": {
      "major": 0.576854066985646,
      "minor": {
        "acc": 0.9431818181818182,
        "f1_micro": 0.2105263157894737,
        "acc_and_f1_micro": 0.576854066985646
      }
    }
  }
}


In [11]:
print(result)

model	task	layer	acc
bert-base-uncased	ner	1	0.9444444444444444
bert-base-uncased	ner	2	0.9259259259259259
bert-base-uncased	ner	3	0.9444444444444444
bert-base-uncased	ner	4	0.9444444444444444
bert-base-uncased	semeval	1	0.7894736842105263
bert-base-uncased	semeval	2	0.8947368421052632
bert-base-uncased	semeval	3	0.868421052631579
bert-base-uncased	semeval	4	0.868421052631579
bert-base-uncased	coref	1	0.6190476190476191
bert-base-uncased	coref	2	0.6190476190476191
bert-base-uncased	coref	3	0.6190476190476191
bert-base-uncased	coref	4	0.5714285714285714
bert-base-uncased	srl	1	0.9242424242424242
bert-base-uncased	srl	2	0.9128787878787878
bert-base-uncased	srl	3	0.9583333333333334
bert-base-uncased	srl	4	0.928030303030303
csarron/bert-base-uncased-squad-v1	ner	1	0.9444444444444444
csarron/bert-base-uncased-squad-v1	ner	2	0.9444444444444444
csarron/bert-base-uncased-squad-v1	ner	3	0.9259259259259259
csarron/bert-base-uncased-squad-v1	ner	4	0.9444444444444444
csarron/bert-base-uncased-squa